In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

import numpy as np
import pandas as pd

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

import warnings
from tqdm import tqdm
import os
from pathlib import Path
# Define the device
device = "cuda" if torch.cuda.is_available() else "mps" if  torch.backends.mps.is_available() else "cpu" # torch.has_mps or
print("Using device:", device)
if (device == 'cuda'):
    print(f"Device name: {torch.cuda.get_device_name(device.index)}")
    print(f"Device memory: {torch.cuda.get_device_properties(device.index).total_memory / 1024 ** 3} GB")
elif (device == 'mps'):
    print(f"Device name: <mps>")
else:
    print("NOTE: If you have a GPU, consider using it for training.")
    print("      On a Windows machine with NVidia GPU, check this video: https://www.youtube.com/watch?v=GMSjDTU8Zlc")
    print("      On a Mac machine, run: pip3 install --pre torch torchvision torchaudio torchtext --index-url https://download.pytorch.org/whl/nightly/cpu")
device = torch.device(device)

Using device: cuda
Device name: Tesla T4
Device memory: 14.74810791015625 GB


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
def run_validation(model, val_loader,loss_fn, device):
    model.eval()
    loss_sum = 0
    num_batch = len(val_loader)
    total_y_true=[]
    total_y_pred=[]
    total_y_prob=[]
    for ESP_val_df_enzy,ESP_val_df_smiles, y_val in val_loader:

        ESP_val_df_enzy = ESP_val_df_enzy.to(device)
        ESP_val_df_smiles = ESP_val_df_smiles.to(device)
        y_val = y_val.squeeze(1).to(device)

        refined_enzy_embed, refined_smiles_embed = model(ESP_val_df_enzy,ESP_val_df_smiles)
        cos_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
        loss = loss_fn(cos_sim, y_val).detach().cpu().numpy()
        loss_sum = loss_sum + loss # count all the loss in the training process
        y_pred = (cos_sim > 0.5).float().cpu().numpy() # if score > 0.5, assign label 1 otherwise 0, transfer to cpu as numpy
        total_y_true.append(y_val.cpu().numpy())
        total_y_pred.append(y_pred)
        total_y_prob.append(cos_sim.detach().cpu().numpy())

    loss_sum = loss_sum/num_batch # get the overall average loss (Notice: this method is not 100% accurate)

    arrange_y_true = np.concatenate(total_y_true, axis=0)
    arrange_y_pred = np.concatenate(total_y_pred, axis=0)
    arrange_y_prob = np.concatenate(total_y_prob, axis=0)
    tn,fp,fn,tp = confusion_matrix(arrange_y_true, arrange_y_pred).ravel()
    acc = (tp+tn)/(tp+tn+fp+fn)
    specificity = tn/(tn+fp)
    sensitivity = tp/(tp+fn)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    bacc = (sensitivity + specificity)/2
    MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    AUC = roc_auc_score(arrange_y_true, arrange_y_prob)
    f1 = 2*precision*recall/(precision+recall)
    print("loss_sum= ",loss_sum, "ACC= ",acc, "bacc= ",bacc, "precision= ",precision,"specificity= ",specificity, "sensitivity= ",sensitivity, "recall= ",recall, "MCC= ",MCC, "AUC= ",AUC, "f1= ",f1)
    return loss_sum, acc, bacc   # , precision, sensitivity, recall, MCC, AUC, f1


## esm 2560 dimension

In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560_phylo')

### pretrain

In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_phylo.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_phylo.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load('ESP_train_df_label_phylo.pt')
    y_val = torch.load('ESP_val_df_label.pt')
    y_test = torch.load('ESP_test_df_label.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 512
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader




In [ ]:
def run_validation(model, val_loader,loss_fn, device):
    model.eval()
    loss_sum = 0
    num_batch = len(val_loader)
    total_y_true=[]
    total_y_pred=[]
    total_y_prob=[]
    for ESP_val_df_enzy,ESP_val_df_smiles, y_val in val_loader:

        ESP_val_df_enzy = ESP_val_df_enzy.to(device)
        ESP_val_df_smiles = ESP_val_df_smiles.to(device)
        y_val = y_val.squeeze(1).to(device)

        refined_enzy_embed, refined_smiles_embed = model(ESP_val_df_enzy,ESP_val_df_smiles)
        cos_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
        loss = loss_fn(cos_sim, y_val).detach().cpu().numpy()
        loss_sum = loss_sum + loss # count all the loss in the training process
        y_pred = (cos_sim > 0.5).float().cpu().numpy() # if score > 0.5, assign label 1 otherwise 0, transfer to cpu as numpy
        total_y_true.append(y_val.cpu().numpy())
        total_y_pred.append(y_pred)
        total_y_prob.append(cos_sim.detach().cpu().numpy())

    loss_sum = loss_sum/num_batch # get the overall average loss (Notice: this method is not 100% accurate)

    arrange_y_true = np.concatenate(total_y_true, axis=0)
    arrange_y_pred = np.concatenate(total_y_pred, axis=0)
    arrange_y_prob = np.concatenate(total_y_prob, axis=0)
    tn,fp,fn,tp = confusion_matrix(arrange_y_true, arrange_y_pred).ravel()
    acc = (tp+tn)/(tp+tn+fp+fn)
    specificity = tn/(tn+fp)
    sensitivity = tp/(tp+fn)
    precision = tp/(tp+fp)
    bacc = (sensitivity + specificity)/2
    recall = tp/(tp+fn)
    MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    AUC = roc_auc_score(arrange_y_true, arrange_y_prob)
    f1 = 2*precision*recall/(precision+recall)
    print(loss_sum, acc, bacc, precision, sensitivity, recall, MCC, AUC, f1)
    return loss_sum, acc, bacc   # , precision, sensitivity, recall, MCC, AUC, f1


In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enzy_refine_layer_1 = nn.Linear(2560, 2560) # W1 and b
        self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(2560, 128) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 128) # W1 and b

        self.relu = nn.ReLU()
        self.batch_norm_enzy = nn.BatchNorm1d(2560)
        self.batch_norm_smiles = nn.BatchNorm1d(768)
        self.batch_norm_shared = nn.BatchNorm1d(128)

    def forward(self, enzy_embed, smiles_embed):
        refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        refined_enzy_embed = self.relu(refined_enzy_embed)
        refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(refined_enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(refined_smiles_embed)

        refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.MSELoss().to(device)

torch.Size([764449, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([764449, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([764449, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
1494


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 500):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")



Epoch 0: 100%|██████████| 1494/1494 [00:26<00:00, 56.81batch/s, train_loss=tensor(0.0205, device='cuda:0', grad_fn=<MseLossBackward0>)]


0.07649002935398709 0.909798110761252 0.8550046353932138 0.901541095890411 0.7389473684210527 0.7389473684210527 0.7600504998538069 0.9432958970148067 0.8121866563825685
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.0765); Accuracy (90.98)


Epoch 1: 100%|██████████| 1494/1494 [00:25<00:00, 58.03batch/s, train_loss=tensor(0.0165, device='cuda:0', grad_fn=<MseLossBackward0>)]


0.06623296388848261 0.9229486942026301 0.8794667973406087 0.908502024291498 0.7873684210526316 0.7873684210526316 0.7964092616668204 0.9544565111823343 0.843609022556391
Epoch: 1 / 500, ############## the best accuracy in val  90.9798 at Epoch: 0  ##############
Performance in Val: Loss: (0.0662); Accuracy (92.29)


Epoch 2: 100%|██████████| 1494/1494 [00:25<00:00, 58.33batch/s, train_loss=tensor(0.0335, device='cuda:0', grad_fn=<MseLossBackward0>)]


...

0.046382798389955 0.948323763659937 0.9138083507712411 0.9584 0.840701754385965 0.840701754385965 0.8648163385003865 0.9581076117571231 0.8957009345794392
Epoch: 497 / 500, ############## the best accuracy in val  95.0917 at Epoch: 376  ##############
Performance in Val: Loss: (0.0464); Accuracy (94.83)


Epoch 498: 100%|██████████| 1494/1494 [00:27<00:00, 54.32batch/s, train_loss=tensor(0.0100, device='cuda:0', grad_fn=<MseLossBackward0>)]


0.04670258408243006 0.9475828857195777 0.9142053170167492 0.9524564183835182 0.8435087719298245 0.8435087719298245 0.8627691165268858 0.9574508869052349 0.8946780796427243
Epoch: 498 / 500, ############## the best accuracy in val  95.0917 at Epoch: 376  ##############
Performance in Val: Loss: (0.0467); Accuracy (94.76)


Epoch 499: 100%|██████████| 1494/1494 [00:27<00:00, 54.01batch/s, train_loss=tensor(0.0107, device='cuda:0', grad_fn=<MseLossBackward0>)]


0.04630809848789464 0.9498055195406557 0.9168404276922806 0.957936507936508 0.8470175438596491 0.8470175438596491 0.8687367750320296 0.9576138761599519 0.8990689013035381
Epoch: 499 / 500, ############## the best accuracy in val  95.0917 at Epoch: 376  ##############
Performance in Val: Loss: (0.0463); Accuracy (94.98)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'

# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560_pretrain_phylo.pt')
run_validation(model_test,val_loader,loss_fn, device)

0.045961483297022904 0.9509168364511946 0.918270512718636 0.9603174603174603 0.8491228070175438 0.8491228070175438 0.8717172139992759 0.9605040659020475 0.9013035381750465


(0.045961483297022904, 0.9509168364511946, 0.918270512718636)

In [ ]:
run_validation(model_test,val_loader,loss_fn, device)
run_validation(model_test,test_loader,loss_fn, device)

0.045961483297022904 0.9509168364511946 0.918270512718636 0.9603174603174603 0.8491228070175438 0.8491228070175438 0.8717172139992759 0.9605040659020475 0.9013035381750465
0.05330231875324479 0.9398287259615384 0.9012593801718469 0.9448275862068966 0.8196581196581196 0.8196581196581196 0.8418809286602347 0.9567112589965068 0.8778032036613271


(0.05330231875324479, 0.9398287259615384, 0.9012593801718469)

### fine tuning
the optimizer state and model will be continue trainin on the smaller train dataset

In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label.pt')
    y_val = torch.load('ESP_val_df_label.pt')
    y_test = torch.load('ESP_test_df_label.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# laod the smaller training dataset
train_loader,  val_loader, test_loader = get_ds()


torch.Size([49881, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([49881, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([49881, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
3118


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 30):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 30,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")



Epoch 0: 100%|██████████| 3118/3118 [00:28<00:00, 108.74batch/s, train_loss=tensor(0.0528, device='cuda:0', grad_fn=<MseLossBackward0>)]


0.04696152170640609 0.9492498610853862 0.9169130929992318 0.9542225730071034 0.848421052631579 0.848421052631579 0.867218291614202 0.9672009288445068 0.8982169390787519
Epoch: 0 / 30, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.0470); Accuracy (94.92)


Epoch 1: 100%|██████████| 3118/3118 [00:28<00:00, 109.32batch/s, train_loss=tensor(0.0414, device='cuda:0', grad_fn=<MseLossBackward0>)]


0.04704041896179902 0.9522133728468235 0.9218519499554119 0.9569303054032889 0.8575438596491228 0.8575438596491228 0.8751086523099209 0.9656173019362699 0.9045151739452257
Epoch: 1 / 30, ############## the best accuracy in val  94.9250 at Epoch: 0  ##############
Performance in Val: Loss: (0.0470); Accuracy (95.22)


Epoch 2: 100%|██████████| 3118/3118 [00:28<00:00, 109.43batch/s, train_loss=tensor(0.0718, device='cuda:0', grad_fn=<MseLossBackward0>)]


...

0.045519232640756545 0.9509168364511946 0.9216464033763321 0.9496124031007752 0.8596491228070176 0.8596491228070176 0.871639360934482 0.969951085564944 0.9023941068139963
Epoch: 27 / 30, ############## the best accuracy in val  95.4621 at Epoch: 19  ##############
Performance in Val: Loss: (0.0455); Accuracy (95.09)


Epoch 28: 100%|██████████| 3118/3118 [00:28<00:00, 108.76batch/s, train_loss=tensor(0.0437, device='cuda:0', grad_fn=<MseLossBackward0>)]


0.0454034915027903 0.9514724949064641 0.9233742130868188 0.947652040030793 0.863859649122807 0.863859649122807 0.8731306982450284 0.9682321051748646 0.9038179148311307
Epoch: 28 / 30, ############## the best accuracy in val  95.4621 at Epoch: 19  ##############
Performance in Val: Loss: (0.0454); Accuracy (95.15)


Epoch 29: 100%|██████████| 3118/3118 [00:28<00:00, 108.72batch/s, train_loss=tensor(0.0293, device='cuda:0', grad_fn=<MseLossBackward0>)]


0.04616451809753129 0.9518429338766439 0.9227256112097052 0.951900698215671 0.8610526315789474 0.8610526315789474 0.8741028783895258 0.9671004511782728 0.9042004421518054
Epoch: 29 / 30, ############## the best accuracy in val  95.4621 at Epoch: 19  ##############
Performance in Val: Loss: (0.0462); Accuracy (95.18)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'

# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560_fine_tuned.pt')
run_validation(model_test,val_loader,loss_fn, device)

0.045488545475128116 0.9546212261529913 0.9264133534641839 0.9573643410852714 0.8666666666666667 0.8666666666666667 0.881493734871767 0.9677255670631031 0.9097605893186005


(0.045488545475128116, 0.9546212261529913, 0.9264133534641839)

In [ ]:
run_validation(model_test,val_loader,loss_fn, device)
run_validation(model_test,test_loader,loss_fn, device)

0.045488545475128116 0.9546212261529913 0.9264133534641839 0.9573643410852714 0.8666666666666667 0.8666666666666667 0.881493734871767 0.9677255670631031 0.9097605893186005
0.05042431509977737 0.9469651442307693 0.9140606225486863 0.9487836107554417 0.8444444444444444 0.8444444444444444 0.8610033733613652 0.9655097279408644 0.8935785348206211


(0.05042431509977737, 0.9469651442307693, 0.9140606225486863)

## esm 1280 dimension

In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_1280_phylo')

### pretrain

In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_phylo.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_phylo.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load('ESP_train_df_label_phylo.pt')
    y_val = torch.load('ESP_val_df_label.pt')
    y_test = torch.load('ESP_test_df_label.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 512
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enzy_refine_layer_1 = nn.Linear(1280, 1280) # W1 and b
        self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(1280, 128) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 128) # W1 and b

        self.relu = nn.ReLU()
        self.batch_norm_enzy = nn.BatchNorm1d(1280)
        self.batch_norm_smiles = nn.BatchNorm1d(768)
        self.batch_norm_shared = nn.BatchNorm1d(128)

    def forward(self, enzy_embed, smiles_embed):
        refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        refined_enzy_embed = self.relu(refined_enzy_embed)
        refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(refined_enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(refined_smiles_embed)

        refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.MSELoss().to(device)

torch.Size([765592, 1280]) torch.Size([5422, 1280]) torch.Size([13336, 1280])
torch.Size([765592, 768]) torch.Size([5422, 768]) torch.Size([13336, 768])
torch.Size([765592, 1]) torch.Size([5422, 1]) torch.Size([13336, 1])
1496


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 500):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")



Epoch 0: 100%|██████████| 1496/1496 [00:21<00:00, 70.45batch/s, train_loss=tensor(0.0316, device='cuda:0', grad_fn=<MseLossBackward0>)]


0.0837222444401546 0.900036886757654 0.8357238447583233 0.89937106918239 0.6995108315863033 0.6995108315863033 0.7327182947777764 0.937650681888897 0.7869496855345911
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.0837); Accuracy (90.00)


Epoch 1: 100%|██████████| 1496/1496 [00:21<00:00, 70.98batch/s, train_loss=tensor(0.0300, device='cuda:0', grad_fn=<MseLossBackward0>)]


0.07062629322436723 0.9192180007377352 0.8687002078926361 0.9182813816343723 0.7617051013277428 0.7617051013277428 0.7859808993933513 0.9518083227443439 0.8326967150496563
Epoch: 1 / 500, ############## the best accuracy in val  90.0037 at Epoch: 0  ##############
Performance in Val: Loss: (0.0706); Accuracy (91.92)


Epoch 2: 100%|██████████| 1496/1496 [00:21<00:00, 70.08batch/s, train_loss=tensor(0.0277, device='cuda:0', grad_fn=<MseLossBackward0>)]


...

0.04882517059079625 0.9444854297307267 0.9087244868389236 0.9505582137161085 0.83298392732355 0.83298392732355 0.8544933953709861 0.9560215061106218 0.8878957169459963
Epoch: 497 / 500, ############## the best accuracy in val  94.6330 at Epoch: 187  ##############
Performance in Val: Loss: (0.0488); Accuracy (94.45)


Epoch 498: 100%|██████████| 1496/1496 [00:22<00:00, 66.36batch/s, train_loss=tensor(0.0055, device='cuda:0', grad_fn=<MseLossBackward0>)]


0.04809494274245067 0.9459609000368867 0.9101749901639276 0.9552 0.8343815513626834 0.8343815513626834 0.8584849705757491 0.9572147919821695 0.8907124207385303
Epoch: 498 / 500, ############## the best accuracy in val  94.6330 at Epoch: 187  ##############
Performance in Val: Loss: (0.0481); Accuracy (94.60)


Epoch 499: 100%|██████████| 1496/1496 [00:22<00:00, 66.58batch/s, train_loss=tensor(0.0065, device='cuda:0', grad_fn=<MseLossBackward0>)]


0.04845934310419993 0.9446698635189967 0.9086256445976193 0.9520383693045563 0.8322851153039832 0.8322851153039832 0.8550028164743516 0.9575145580000844 0.8881431767337807
Epoch: 499 / 500, ############## the best accuracy in val  94.6330 at Epoch: 187  ##############
Performance in Val: Loss: (0.0485); Accuracy (94.47)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'

# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_1280_pretrain_phylo.pt')
run_validation(model_test,val_loader,loss_fn, device)

0.04879631610079245 0.9463297676134268 0.9108738021834943 0.9552715654952076 0.8357791754018169 0.8357791754018169 0.8594664642632134 0.9575208614911154 0.8915393216548639


(0.04879631610079245, 0.9463297676134268, 0.9108738021834943)

In [ ]:
run_validation(model_test,val_loader,loss_fn, device)
run_validation(model_test,test_loader,loss_fn, device)

0.04879631610079245 0.9463297676134268 0.9108738021834943 0.9552715654952076 0.8357791754018169 0.8357791754018169 0.8594664642632134 0.9575208614911154 0.8915393216548639
0.053451908855802484 0.9392621475704859 0.8997839379594939 0.9459459459459459 0.8162684869169511 0.8162684869169511 0.8403630591132002 0.9533958088598179 0.8763358778625955


(0.053451908855802484, 0.9392621475704859, 0.8997839379594939)

### fine tuning
the optimizer state and model will be continue trainin on the smaller train dataset

In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label.pt')
    y_val = torch.load('ESP_val_df_label.pt')
    y_test = torch.load('ESP_test_df_label.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# laod the smaller training dataset
train_loader,  val_loader, test_loader = get_ds()


torch.Size([50077, 1280]) torch.Size([5422, 1280]) torch.Size([13336, 1280])
torch.Size([50077, 768]) torch.Size([5422, 768]) torch.Size([13336, 768])
torch.Size([50077, 1]) torch.Size([5422, 1]) torch.Size([13336, 1])
3130


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 30):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 30,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")



Epoch 0: 100%|██████████| 3130/3130 [00:23<00:00, 135.24batch/s, train_loss=tensor(0.1215, device='cuda:0', grad_fn=<MseLossBackward0>)]


0.048960982040285095 0.9472519365547768 0.9141897011112179 0.950432730133753 0.8441649196366178 0.8441649196366178 0.8618681470996554 0.9651142919227241 0.8941524796447076
Epoch: 0 / 30, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.0490); Accuracy (94.73)


Epoch 1: 100%|██████████| 3130/3130 [00:23<00:00, 134.67batch/s, train_loss=tensor(0.0532, device='cuda:0', grad_fn=<MseLossBackward0>)]


0.04785917406238253 0.9490962744374769 0.9158907682047011 0.9565217391304348 0.8455625436757512 0.8455625436757512 0.8668314934480938 0.966115846608748 0.897626112759644
Epoch: 1 / 30, ############## the best accuracy in val  94.7252 at Epoch: 0  ##############
Performance in Val: Loss: (0.0479); Accuracy (94.91)


Epoch 2: 100%|██████████| 3130/3130 [00:23<00:00, 133.68batch/s, train_loss=tensor(0.0326, device='cuda:0', grad_fn=<MseLossBackward0>)]


...

0.047697950316459806 0.9513094798967171 0.9216525092009082 0.9519752130131681 0.8588399720475192 0.8588399720475192 0.8726792443722489 0.9663301653037994 0.9030124908155768
Epoch: 27 / 30, ############## the best accuracy in val  95.2416 at Epoch: 23  ##############
Performance in Val: Loss: (0.0477); Accuracy (95.13)


Epoch 28: 100%|██████████| 3130/3130 [00:23<00:00, 130.86batch/s, train_loss=tensor(0.0424, device='cuda:0', grad_fn=<MseLossBackward0>)]


0.04644727348528561 0.951493913684987 0.9215536669596038 0.953416149068323 0.8581411600279525 0.8581411600279525 0.8731731650712458 0.9690732344840882 0.9032732622287606
Epoch: 28 / 30, ############## the best accuracy in val  95.2416 at Epoch: 23  ##############
Performance in Val: Loss: (0.0464); Accuracy (95.15)


Epoch 29: 100%|██████████| 3130/3130 [00:23<00:00, 132.51batch/s, train_loss=tensor(0.0192, device='cuda:0', grad_fn=<MseLossBackward0>)]


0.04712645924007273 0.950387310955367 0.9210260997797104 0.9483024691358025 0.8588399720475192 0.8588399720475192 0.8702270582377633 0.9668934522662014 0.9013568023469014
Epoch: 29 / 30, ############## the best accuracy in val  95.2416 at Epoch: 23  ##############
Performance in Val: Loss: (0.0471); Accuracy (95.04)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'

# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_1280_fine_tuned.pt')
run_validation(model_test,val_loader,loss_fn, device)

0.0468038842183329 0.9524160826263371 0.9228524487574331 0.9550038789759504 0.8602375960866527 0.8602375960866527 0.8756294607388913 0.9663028501759987 0.9051470588235294


(0.0468038842183329, 0.9524160826263371, 0.9228524487574331)

In [ ]:
run_validation(model_test,val_loader,loss_fn, device)
run_validation(model_test,test_loader,loss_fn, device)

0.0468038842183329 0.9524160826263371 0.9228524487574331 0.9550038789759504 0.8602375960866527 0.8602375960866527 0.8756294607388913 0.9663028501759987 0.9051470588235294
0.05134982314188038 0.9455608878224355 0.9132812699118837 0.9425761421319797 0.8449943117178612 0.8449943117178612 0.8572423068955304 0.9634896423449162 0.891121775644871


(0.05134982314188038, 0.9455608878224355, 0.9132812699118837)